In [1]:
import pandas as pd
import numpy as np

In [2]:
#Leo el csv de eventos

events_df = pd.read_csv("/home/miki_mustard/Desktop/Facultad/Datos/TP2/fiuba-trocafone-tp2-final-set/events_up_to_01062018.csv", low_memory=False)

## Generacion de features

In [3]:
features_df = pd.DataFrame()

In [4]:
features_df['person'] = events_df.person.unique()

In [5]:
features_df.head()

,person
0,4886f805
1,ad93850f
2,0297fc1e
3,2d681dd8
4,cccea85e


## Cantidad eventos totales por usuario

In [6]:
df = events_df.groupby('person').event.count()

In [7]:
features_df = features_df.join(df, on='person')

In [8]:
features_df.rename(columns={'event':'cant_eventos_totales'}, inplace=True)

## Cantidad eventos de cada uno

In [53]:
df = events_df.groupby('person').event.value_counts()

In [54]:
df = df.unstack()
df.head()

event,ad campaign hit,brand listing,checkout,conversion,generic listing,lead,search engine hit,searched products,staticpage,viewed product,visited site
person,,,,,,,,,,,
0008ed71,NaN,NaN,3.0,NaN,1.0,NaN,NaN,NaN,NaN,NaN,2.0
00091926,15.0,25.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,372.0,34.0
00091a7a,1.0,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,1.0
000ba417,1.0,24.0,6.0,1.0,14.0,NaN,1.0,NaN,NaN,153.0,6.0
000c79fe,1.0,NaN,1.0,NaN,1.0,NaN,1.0,9.0,NaN,3.0,1.0


In [55]:
df = df.drop(columns=['lead'])

In [56]:
df['ach*bl'] = (df['ad campaign hit'])*(df['brand listing'])
df['ach*gl'] = (df['ad campaign hit'])*(df['generic listing'])
df['ach*seh'] = (df['ad campaign hit'])*(df['search engine hit'])
df['ach*sp'] = (df['ad campaign hit'])*(df['searched products'])
df['ach*vp'] = (df['ad campaign hit'])*(df['viewed product'])
df['ach*vs'] = (df['ad campaign hit'])*(df['visited site'])

In [58]:
df.columns

Index(['ad campaign hit', 'brand listing', 'checkout', 'conversion',
       'generic listing', 'search engine hit', 'searched products',
       'staticpage', 'viewed product', 'visited site', 'ach*bl', 'ach*gl',
       'ach*seh', 'ach*sp', 'ach*vp', 'ach*vs'],
      dtype='object', name='event')

In [61]:
df = df.drop(columns=['checkout', 'conversion', 'visited site', 'staticpage', 'generic listing']).head()

In [62]:
features_df = features_df.join(df, on='person')

In [ ]:
features_df.rename(columns={'ad campaign hit':'cant_ach', 'brand listing':'cant_bl', 'checkout':'cant_ch', 
                            'conversion':'cant_cv', 'generic listing':'cant_gl',
                           'search engine hit':'cant_seh', 'searched products':'cant_s_p', 
                            'staticpage':'cant_sp', 'viewed product':'cant_vp', 'visited site':'cant_vs'}, inplace=True)

## Quincenas

In [9]:
events_df['timestamp'] = pd.to_datetime(events_df['timestamp'])
events_df['mes'] = events_df['timestamp'].dt.month
events_df['dia'] = events_df['timestamp'].dt.day

In [10]:
events_df['q1'] = ((events_df['mes']==1) & (events_df['dia']>1) & (events_df['dia']<16))
events_df['q2'] = ((events_df['mes']==1) & (events_df['dia']>15) & (events_df['dia']<32))
events_df['q3'] = ((events_df['mes']==2) & (events_df['dia']>1) & (events_df['dia']<16))
events_df['q4'] = ((events_df['mes']==2) & (events_df['dia']>15) & (events_df['dia']<29))
events_df['q5'] = ((events_df['mes']==3) & (events_df['dia']>1) & (events_df['dia']<16))
events_df['q6'] = ((events_df['mes']==3) & (events_df['dia']>15) & (events_df['dia']<32))
events_df['q7'] = ((events_df['mes']==4) & (events_df['dia']>1) & (events_df['dia']<16))
events_df['q8'] = ((events_df['mes']==4) & (events_df['dia']>15) & (events_df['dia']<31))
events_df['q9'] = ((events_df['mes']==5) & (events_df['dia']>1) & (events_df['dia']<16))
events_df['q10'] = ((events_df['mes']==5) & (events_df['dia']>15) & (events_df['dia']<32))

In [11]:
df = events_df[['person', 'q1','q2','q3','q4','q5','q6','q7','q8','q9','q10']].groupby('person').agg(lambda x: x.any())
df.head()

,q1,q2,q3,q4,q5,q6,q7,q8,q9,q10
person,,,,,,,,,,
0008ed71,False,False,False,False,False,False,False,False,False,True
00091926,False,False,False,False,False,False,False,False,True,True
00091a7a,False,False,False,False,False,True,False,False,False,False
000ba417,False,False,False,False,False,False,False,False,False,True
000c79fe,False,False,False,False,False,False,False,False,False,True


In [12]:
features_df = features_df.join(df, on='person')

## dia con mas entradas

In [13]:
# Los mas frecuentes son martes, miercoles y jueves
events_df['weekday'] = events_df['timestamp'].dt.day_name()

In [14]:
df = events_df.groupby('person').weekday.value_counts()

In [15]:
df.unstack().drop(columns=['Monday', 'Saturday', 'Sunday', 'Friday']).head()

weekday,Thursday,Tuesday,Wednesday
person,,,
0008ed71,6.0,NaN,NaN
00091926,84.0,43.0,40.0
00091a7a,NaN,NaN,NaN
000ba417,147.0,NaN,NaN
000c79fe,NaN,17.0,NaN


In [16]:
df = df.unstack().drop(columns=['Monday', 'Saturday', 'Sunday', 'Friday'])
df.head()

weekday,Thursday,Tuesday,Wednesday
person,,,
0008ed71,6.0,NaN,NaN
00091926,84.0,43.0,40.0
00091a7a,NaN,NaN,NaN
000ba417,147.0,NaN,NaN
000c79fe,NaN,17.0,NaN


In [17]:
features_df = features_df.join(df, on='person')

In [63]:
#Tiro las columnas con menos de 0.04 de importance
features_df = features_df.drop(columns=['q3', 'q1', 'q7', 'q6', 'q5', 'q4', 'q2'])

In [18]:
labels_df = pd.read_csv("/home/miki_mustard/Desktop/Facultad/Datos/TP2/fiuba-trocafone-tp2-final-set/labels_training_set.csv", low_memory=False)

In [89]:
#Separo en train y to_predict
train_df = features_df.merge(labels_df, on='person', how='right')
train_df.shape

(19414, 9)

In [90]:
train_df.fillna(0, inplace=True)

In [91]:
to_predict = features_df[~features_df.person.isin(labels_df.person)]
to_predict.shape

(19415, 8)

In [92]:
to_predict.fillna(0, inplace=True)

/home/miki_mustard/envdatos/lib/python3.6/site-packages/pandas/core/frame.py:3790: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  downcast=downcast, **kwargs)


In [93]:
# Labels: los valores a predecir
#labels = np.array(train_df.label)
labels = train_df.label

#remuevo el label del df
train_df = train_df.drop(columns=['label'])

In [94]:
# Using Skicit-learn to split data into training and testing sets
from sklearn.model_selection import train_test_split

#Separo en test y train
X_train, X_test, y_train, y_test = \
    train_test_split(train_df.drop(columns=['person']) , labels, test_size=0.2, random_state=123)

In [95]:
X_train.reset_index(inplace=True)
X_train.drop(columns='index', inplace=True)
X_test.reset_index(inplace=True)
X_test.drop(columns='index', inplace=True)
y_train = pd.DataFrame(y_train)

y_train.reset_index(inplace=True)
y_train.drop(columns='index', inplace=True)
y_test = pd.DataFrame(y_test)
y_test.reset_index(inplace=True)
y_test.drop(columns='index', inplace=True)

to_predict = pd.DataFrame(to_predict)
to_predict.reset_index(inplace=True)
to_predict.drop(columns='index', inplace=True)

## SVM

In [ ]:
from sklearn.svm import LinearSVR

In [ ]:
regr = LinearSVR(random_state=0, tol=1e-5)

In [ ]:
regr.fit(X_train, y_train)

In [ ]:
preds = regr.predict(X_test)

In [ ]:
preds

In [ ]:
from sklearn.metrics import roc_auc_score
roc_auc_score(y_test, preds)

In [ ]:
from sklearn import linear_model
n_samples, n_features = 10, 5

clf = linear_model.SGDRegressor(max_iter=1000, tol=1e-3)
clf.fit(X_train, y_train)

In [ ]:
preds = clf.predict(X_test)

In [ ]:
roc_auc_score(y_test, preds)

In [ ]:
roc_auc_score(y_test, preds)

## Random Forest

In [96]:
# Guardo los nombres de los features para usarlos luego
feature_list = list(train_df.drop(columns=['person']).columns)

In [97]:
#Train model

# Import the model we are using
from sklearn.ensemble import RandomForestRegressor

# Instantiate model with 1000 decision trees
rf = RandomForestRegressor(n_estimators = 1000, random_state = 42)

# Train the model on training data
rf.fit(X_train, y_train.label);

In [98]:
# Use the forest's predict method on the test data
predictions = rf.predict(X_test)

In [99]:
from sklearn.metrics import roc_auc_score
roc_auc_score(y_test,predictions)

0.6275254366943156

In [75]:
roc_auc_score(y_test,predictions)

0.6280132637085873

In [100]:
#Obtengo la importancia de los features

# Get numerical feature importances
importances = list(rf.feature_importances_)

# List of tuples with variable and importance
feature_importances = [(feature, round(importance, 2)) for feature, importance in zip(feature_list, importances)]

# Sort the feature importances by most important first
feature_importances = sorted(feature_importances, key = lambda x: x[1], reverse = True)

# Print out the feature and importances 
[print('Variable: {:20} Importance: {}'.format(*pair)) for pair in feature_importances];

Variable: cant_eventos_totales Importance: 0.34
Variable: Thursday             Importance: 0.19
Variable: Tuesday              Importance: 0.19
Variable: Wednesday            Importance: 0.19
Variable: q9                   Importance: 0.04
Variable: q8                   Importance: 0.03
Variable: q10                  Importance: 0.02


In [88]:
features_df = features_df.drop(columns=['ach*vs', 'ach*vp', 'ach*sp', 'ach*seh', 'ach*gl', 'ach*bl', 'viewed product', 'searched products', 'search engine hit', 'brand listing', 'ad campaign hit'])

In [38]:
rf = RandomForestRegressor()
from sklearn.model_selection import GridSearchCV
#Grid search
rf_param_grid = {
    'max_depth':[None],
    'max_features':[1,3,10],
    #'min_sample_split':[2,3],
    'min_samples_leaf':[1,3,10],
    'n_estimators':[50,75,100],
}

rf_gs = GridSearchCV(rf, param_grid=rf_param_grid, scoring='roc_auc', n_jobs=4, refit=True, cv=2, verbose=1, return_train_score=True)

In [39]:
rf_gs.fit(X_train, y_train)

Fitting 2 folds for each of 27 candidates, totalling 54 fits


[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:   10.2s
[Parallel(n_jobs=4)]: Done  54 out of  54 | elapsed:   13.6s finished
/home/miki_mustard/envdatos/lib/python3.6/site-packages/sklearn/model_selection/_search.py:740: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self.best_estimator_.fit(X, y, **fit_params)


GridSearchCV(cv=2, error_score='raise-deprecating',
       estimator=RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators='warn', n_jobs=None,
           oob_score=False, random_state=None, verbose=0, warm_start=False),
       fit_params=None, iid='warn', n_jobs=4,
       param_grid={'max_depth': [None], 'max_features': [1, 3, 10], 'min_samples_leaf': [1, 3, 10], 'n_estimators': [50, 75, 100]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring='roc_auc', verbose=1)

In [40]:
rf_gs.best_estimator_

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features=1, max_leaf_nodes=None, min_impurity_decrease=0.0,
           min_impurity_split=None, min_samples_leaf=10,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           n_estimators=100, n_jobs=None, oob_score=False,
           random_state=None, verbose=0, warm_start=False)

In [41]:
rf_gs.best_score_

0.6943192205477366

In [42]:
preds_rf_gs = rf_gs.predict(to_predict.drop(columns=['person']))

## XGBoost

In [43]:
import xgboost as xgb
from xgboost import XGBRegressor
#Parametros con los que obtuvimos 0.82

xg_reg = xgb.XGBRegressor(objective ='binary:logistic',base_score= 0.1, 
                          colsample_bylevel= 0.9, booster='dart', 
                colsample_bytree = 0.5, learning_rate = 0.2,
                max_depth = 8, alpha = 20, gamma=10, n_estimators = 35, eval_metric='auc')

In [101]:
xg_reg.fit(X_train,y_train)

XGBRegressor(alpha=20, base_score=0.1, booster='dart', colsample_bylevel=0.9,
       colsample_bytree=0.5, eval_metric='auc', gamma=10,
       learning_rate=0.2, max_delta_step=0, max_depth=8,
       min_child_weight=1, missing=None, n_estimators=35, n_jobs=1,
       nthread=None, objective='binary:logistic', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1)

In [102]:
preds = xg_reg.predict(X_test)

In [103]:
roc_auc_score(y_test,preds)

0.7008323852309967

In [80]:
roc_auc_score(y_test,preds)

0.6982757502973962

In [48]:
from sklearn.model_selection import GridSearchCV

In [81]:
param_grid =  {
        'min_child_weight': [1, 5, 10],
        'gamma': [1, 1.5, 10],
        'colsample_bytree': [0.1, 0.2, 0.3],
        'colsample_bylevel': [0.8, 0.9],
        'max_depth': [9, 11, 13],
        'base_score': [0.1, 0.2],
        'n_estimators': [50, 70, 80, 90]
        }

In [82]:
clf = GridSearchCV(xg_reg, param_grid, scoring='roc_auc', n_jobs=4, refit=True, cv=3, verbose=3, return_train_score=True)
clf.fit(X_train, y_train)

Fitting 3 folds for each of 1296 candidates, totalling 3888 fits


[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  24 tasks      | elapsed:    4.8s
[Parallel(n_jobs=4)]: Done 120 tasks      | elapsed:   24.3s
[Parallel(n_jobs=4)]: Done 280 tasks      | elapsed:   50.0s
[Parallel(n_jobs=4)]: Done 504 tasks      | elapsed:  1.6min
[Parallel(n_jobs=4)]: Done 792 tasks      | elapsed:  3.1min
[Parallel(n_jobs=4)]: Done 1144 tasks      | elapsed:  4.7min
[Parallel(n_jobs=4)]: Done 1560 tasks      | elapsed:  6.1min
[Parallel(n_jobs=4)]: Done 2040 tasks      | elapsed:  8.6min
[Parallel(n_jobs=4)]: Done 2584 tasks      | elapsed: 10.4min
[Parallel(n_jobs=4)]: Done 3192 tasks      | elapsed: 13.2min
[Parallel(n_jobs=4)]: Done 3864 tasks      | elapsed: 16.3min
[Parallel(n_jobs=4)]: Done 3888 out of 3888 | elapsed: 16.4min finished


GridSearchCV(cv=3, error_score='raise-deprecating',
       estimator=XGBRegressor(alpha=20, base_score=0.1, booster='dart', colsample_bylevel=0.9,
       colsample_bytree=0.5, eval_metric='auc', gamma=10,
       learning_rate=0.2, max_delta_step=0, max_depth=8,
       min_child_weight=1, missing=None, n_estimators=35, n_jobs=1,
       nthread=None, objective='binary:logistic', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1),
       fit_params=None, iid='warn', n_jobs=4,
       param_grid={'min_child_weight': [1, 5, 10], 'gamma': [1, 1.5, 10], 'colsample_bytree': [0.1, 0.2, 0.3], 'colsample_bylevel': [0.8, 0.9], 'max_depth': [9, 11, 13], 'base_score': [0.1, 0.2], 'n_estimators': [50, 70, 80, 90]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring='roc_auc', verbose=3)

In [84]:
clf.best_estimator_

XGBRegressor(alpha=20, base_score=0.2, booster='dart', colsample_bylevel=0.8,
       colsample_bytree=0.3, eval_metric='auc', gamma=10,
       learning_rate=0.2, max_delta_step=0, max_depth=13,
       min_child_weight=1, missing=None, n_estimators=50, n_jobs=1,
       nthread=None, objective='binary:logistic', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1)

In [85]:
clf.best_score_

0.6896904225828608

In [86]:
preds_gs = clf.predict(X_test)

In [87]:
roc_auc_score(y_test, preds_gs)

0.6995893177228689

In [ ]:
preds_xgb = xg_reg.predict(to_predict.drop(columns=['person']))

In [ ]:
preds_xgb.shape

In [ ]:
preds_xgb

In [ ]:
preds_rf_gs.shape

In [ ]:
combinado_xgb_y_rf = preds_rf_gs*0.2 + preds_xgb*0.8
combinado_xgb_y_rf

In [ ]:
roc_auc_score(y_test, combinado_xgb_y_rf)

## Logistic Regression

In [ ]:
from sklearn.linear_model import LogisticRegression

In [ ]:
params ={
    'C':[0.8, 0.9, 1.0],
    'max_iter':[50, 70, 100],
    'n_jobs':[1, 2],
    'random_state':[1,2,3]
}

In [ ]:
l_reg = LogisticRegression(random_state=0, solver='sag')

In [ ]:
clf = GridSearchCV(l_reg, params, scoring='roc_auc', n_jobs=4, refit=True, cv=3, verbose=3, return_train_score=True)

In [ ]:
clf.fit(X_train, y_train)

In [ ]:
preds = clf.predict_proba(X_test)[:,1] 

In [ ]:
clf.score(X_test, y_test)

In [ ]:
roc_auc_score(y_test, preds)

## CSV a publicar

In [ ]:
to_publish = pd.DataFrame()

In [ ]:
to_publish['person'] = to_predict.person
to_publish.shape

In [ ]:
to_publish['label'] = combinado_xgb_y_rf
to_publish.shape

In [ ]:
to_publish.head()

In [ ]:
to_publish.loc[ to_publish.label > 1 ]

In [ ]:
to_publish.loc[ to_publish.label < 0 ]

In [ ]:
to_publish.to_csv('29_11_2.csv', index=False)